In [2]:
# get rid of the annoying ipython import cache
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
# Python
from __future__ import annotations
# Internal
from research.core.sources.dbms import DataBase

## Initial Loading
First we need to connect to the DB and get the data.
I created a small wrapper for the Python Postgres driver.
This wrapper adds a few 'syntactic sugar' methods to the interface + session control.

In [4]:
database = DataBase()

Get the first feeling of the data we have in the tables.

In [5]:
# Explore the tables
database.read('SELECT * FROM listings LIMIT 1', dataframe=True)

,web_id,meta_valid_from_dts,meta_valid_to_dts,feed_last_update,offering_type_id,property_type_id,bedroom_id,bathroom_id,property_sqft,property_cheques,property_price,transacted_after_a_month,location_path_name_primary,coordinates_lat,coordinates_lon
0,6406935,2018-10-27 17:36:30.000,2018-11-05 11:00:21.000,,1,1,3,4,1362,None,2929842,0,"Dubai,Downtown Dubai,The Address Residence Fou...",25.1949,55.2827


In [6]:
database.read('SELECT * FROM poi LIMIT 1', dataframe=True)

,name,category,lat,lon
0,Expo 2020 Site,Landmark,24.9606,55.1506


The data is very tiny. Only ~61k rows in total. I prefer to load data directly in memory and work with it.
However, for BigData (not fitting into RAM) I would be using absolutely different approach:
Lazy Loading with dask/pyspark.
Also, I would like to rename column, because the names are suboptimal.

In [7]:
listings = database.read("""
SELECT  web_id ID, meta_valid_from_dts valid_from, meta_valid_to_dts valid_to,
        feed_last_update updated, offering_type_id offer_type, property_type_id property_type,
        bedroom_id bedrooms, bathroom_id bathrooms, property_sqft sqft, property_cheques cheques,
        property_price price, transacted_after_a_month transacted,
        location_path_name_primary location_path, coordinates_lat latitude, coordinates_lon longitude
FROM    listings""", dataframe=True)
listings

,id,valid_from,valid_to,updated,offer_type,property_type,bedrooms,bathrooms,sqft,cheques,price,transacted,location_path,latitude,longitude
0,6406935,2018-10-27 17:36:30.000,2018-11-05 11:00:21.000,,1,1,3,4,1362,NaN,2929842,0.0,"Dubai,Downtown Dubai,The Address Residence Fou...",25.1949,55.2827
1,5949553,2018-10-30 13:01:41.000,2019-02-14 08:00:31.000,,1,1,7,7,16000,NaN,12500000,0.0,"Dubai,Al Barari,Al Barari Villas",25.0981,55.3106
2,6389258,2018-10-30 07:01:43.000,2018-11-01 16:00:59.000,,1,1,6,6,3600,NaN,3227499,0.0,"Dubai,Jumeirah Park,Legacy",25.0481,55.1544
3,6467389,2018-10-27 17:46:52.000,2018-12-12 10:00:43.000,,2,1,2,3,981,12.0,88400,0.0,"Dubai,Culture Village,Dubai Wharf",25.2257,55.3387
4,6479131,2018-10-27 17:36:30.000,2018-11-20 11:33:25.000,,1,1,2,3,860,NaN,1100000,0.0,"Dubai,Umm Suqeim,Madinat Jumeirah Living,Lamtara",25.1374,55.1890
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59995,6192930,2018-10-27 17:46:52.000,2018-11-05 13:01:16.000,,1,1,6,5,4909,NaN,4679999,0.0,"Dubai,Arabian Ranches,Saheel,Saheel 3",25.0561,55.2667
59996,6477383,2018-10-27 17:36:30.000,2018-11-20 11:42:05.000,,2,1,4,4,1900,NaN,165000,0.0,"Dubai,Dubai Marina,Al Majara,Al Majara 1",25.0742,55.1378
59997,6396153,2018-10-31 06:01:08.000,2018-11-20 11:33:25.000,,2,1,2,2,1603,NaN,167283,0.0,"Dubai,Jumeirah Beach Residence,Shams,Shams 4",25.0728,55.1298
59998,6367449,2018-10-27 17:46:52.000,2018-11-01 14:10:56.000,,2,1,3,4,1670,NaN,195455,0.0,"Dubai,Jumeirah Beach Residence,The Walk,Al Bat...",25.0730,55.1281


In [8]:
poi = database.read("""
SELECT  name, category, lat latitude, lon longitude
FROM    poi
""", dataframe=True)
poi

,name,category,latitude,longitude
0,Expo 2020 Site,Landmark,24.9606,55.1506
1,Marina Mall,Landmark,25.0765,55.1407
2,Hamdan Sports Complex,Landmark,25.0512,55.3181
3,Motor City,Landmark,25.0449,55.2395
4,Jabel Ali,Landmark,25.0034,54.9904
...,...,...,...,...
72,BurJuman,Metro,25.2550,55.3040
73,Mall of the Emirates,Metro,25.1210,55.2000
74,Oud Metha,Metro,25.2440,55.3160
75,Palm Deira,Metro,25.2760,55.3010


## First Look
Check how much information every column gives to us.
If column consists of only 1 unique value -> there is no info.

In [9]:
uniques = listings.nunique().to_frame(name='uniques')
uniques['share_%'] = (uniques['uniques'] / listings.shape[0] * 100).round(2)
uniques

,uniques,share_%
id,60000,100.00
valid_from,615,1.03
valid_to,2405,4.01
updated,1,0.00
offer_type,2,0.00
property_type,1,0.00
bedrooms,10,0.02
bathrooms,9,0.02
sqft,5128,8.55
cheques,10,0.02


Use id as index, since it's unique and is not gonna be used as feature

In [10]:
listings.set_index(keys=['id'], drop=True, inplace=True)

Drop updated and property_type. They have only one unique value, not informative.

In [11]:
listings.drop(columns=['updated', 'property_type'], inplace=True)

## Optimize
Optimization of datatypes reduces overall dataframe size and increases operation speed.
Offtop: This is very important step, when we are dealing with some bigger sets (e.g. 4Gb),
that still fit in RAM but are not considered as Big Data. Usually,
optimization reduces DF's memory consumption by ~40-60%

In [12]:
dtypes = {'valid_from': 'datetime64[s]',
          'valid_to': 'datetime64[s]',
          'offer_type': 'uint8',
          'bedrooms': 'uint8',
          'bathrooms': 'uint8',
          'sqft': 'uint32',
          'price': 'uint64',
          'transacted': 'float32', # Nullable
          'location_path': 'str',
          'latitude': 'float32',
          'longitude': 'float32'}
listings = listings.astype(dtypes)
listings

,valid_from,valid_to,offer_type,bedrooms,bathrooms,sqft,cheques,price,transacted,location_path,latitude,longitude
id,,,,,,,,,,,,
6406935,2018-10-27 17:36:30,2018-11-05 11:00:21,1,3,4,1362,NaN,2929842,0.0,"Dubai,Downtown Dubai,The Address Residence Fou...",25.194901,55.282700
5949553,2018-10-30 13:01:41,2019-02-14 08:00:31,1,7,7,16000,NaN,12500000,0.0,"Dubai,Al Barari,Al Barari Villas",25.098101,55.310600
6389258,2018-10-30 07:01:43,2018-11-01 16:00:59,1,6,6,3600,NaN,3227499,0.0,"Dubai,Jumeirah Park,Legacy",25.048100,55.154400
6467389,2018-10-27 17:46:52,2018-12-12 10:00:43,2,2,3,981,12.0,88400,0.0,"Dubai,Culture Village,Dubai Wharf",25.225700,55.338699
6479131,2018-10-27 17:36:30,2018-11-20 11:33:25,1,2,3,860,NaN,1100000,0.0,"Dubai,Umm Suqeim,Madinat Jumeirah Living,Lamtara",25.137400,55.188999
...,...,...,...,...,...,...,...,...,...,...,...,...
6192930,2018-10-27 17:46:52,2018-11-05 13:01:16,1,6,5,4909,NaN,4679999,0.0,"Dubai,Arabian Ranches,Saheel,Saheel 3",25.056101,55.266701
6477383,2018-10-27 17:36:30,2018-11-20 11:42:05,2,4,4,1900,NaN,165000,0.0,"Dubai,Dubai Marina,Al Majara,Al Majara 1",25.074200,55.137798
6396153,2018-10-31 06:01:08,2018-11-20 11:33:25,2,2,2,1603,NaN,167283,0.0,"Dubai,Jumeirah Beach Residence,Shams,Shams 4",25.072800,55.129799


In [13]:
dtypes = {'name': 'str',
          'category': 'str',
          'latitude': 'float32',
          'longitude': 'float32'}
poi = poi.astype(dtypes)
poi

,name,category,latitude,longitude
0,Expo 2020 Site,Landmark,24.960600,55.150600
1,Marina Mall,Landmark,25.076500,55.140701
2,Hamdan Sports Complex,Landmark,25.051201,55.318100
3,Motor City,Landmark,25.044901,55.239498
4,Jabel Ali,Landmark,25.003401,54.990398
...,...,...,...,...
72,BurJuman,Metro,25.254999,55.304001
73,Mall of the Emirates,Metro,25.121000,55.200001
74,Oud Metha,Metro,25.243999,55.316002
75,Palm Deira,Metro,25.275999,55.300999


## Results and Dump
What we got so far: cleaner, optimized, informative dataset.
Dump the result and move to the Preprocessing step.

In [14]:
listings.to_pickle('core/loading/listings.pkl.gzip', protocol=4)
poi.to_pickle('core/loading/poi.pkl.gzip', protocol=4)